In [1]:
cd ..

/Users/kylemagida/git/blobs


In [2]:
# Import required libraries and assign root directory of blobs to root
import blobs
import numpy as np
import pandas as pd
import pysal as ps
import os
from shapely import geometry
import geopandas as gpd #Build most recent version from Git, don't use pip install, libspatialindex and Rtree are dependencies as well
from geopandas.tools import sjoin

root = os.getcwd()
%pylab inline
SHAPE_FILE = root + '/tracts/CensusTractsTIGER2010.shp'
DBF_FILE = root + '/tracts/CensusTractsTIGER2010.dbf'
OUTPUT_FILE = root + '/test_data.csv'

Your version of PySAL is 185 days old.
There have likely been 1 new release(s).
Disable this check? [Y/n]n
Populating the interactive namespace from numpy and matplotlib


In [3]:
#Load shapefiles into geopandas and convert to the correct projection

POINT_PROJECTION = 4326 #Projection system of points, 4326 is traditional lat/long
GEOMETRY = 'geometry' # Location of geometry in the shapefile database
GEO_ID = 'TRACTCE10' # ID in the shapefile to be aggregated on, can be changed to blocks or tracts
DEFAULT_CRS = {'datum': 'NAD83','k': 0.999975, 'lat_0': 36.66666666666666,'lon_0': -88.33333333333333,
 'no_defs': True,'proj': u'tmerc',u'units': u'us-ft',u'x_0': 300000, u'y_0': 0} # Default CRS assuming census tiger tracts with NAD83 projections

shape_df = gpd.read_file(SHAPE_FILE)
if shape_df.crs == {}:
    shape_df = gpd.GeoDataFrame(shape_df,crs=DEFAULT_CRS)
shape_df[GEOMETRY] = shape_df[GEOMETRY].to_crs(epsg=POINT_PROJECTION)
shape_df = gpd.GeoDataFrame(shape_df,crs=None) #Remove crs to complete join

In [9]:
# Run through each file in filelist and build a final dataframe with points from all the specified files, each column with the given header

# This filelist can be built more dynamically if we want to rely on people pulling from Plenario
filelist = [{'location':'/Users/kylemagida/Downloads/business_licenses_2015-09-06.csv','lat':'latitude','long':'longitude',
             'unique_id':'business_licenses_row_id','name':'biz1'}]

init_calls = pd.DataFrame()

for file_dict in filelist:
    unique_id = file_dict['unique_id']
    point_df = build_new_points(file_dict)
    final_df = points_in_shapes(point_df,shape_df,file_dict)
    grouped = final_df.groupby(by=GEO_ID).count()[unique_id]
    grouped = pd.DataFrame(grouped).rename(columns={unique_id:file_dict['name']})
    init_calls = init_calls.join(grouped,how='outer')

for c in init_calls.columns:
    init_calls[c] = init_calls[c].astype('float')
#init_calls[GEO_ID] = init_calls.index
init_calls.rename(columns={GEO_ID:'tractce10'},inplace=True)
init_calls.to_csv(OUTPUT_FILE,index_label='tractce10')

In [4]:
# Functions to use to log the associated tracts of a given set of points
def points_in_shapes(point_df,shape_df,file_dict):
    '''
    Joines points and shapes and returns the joined dataframe
    '''
    active_points_df = point_out_of_bounds(point_df,shape_df,file_dict)
    join_left_df = sjoin(active_points_df, shape_df, how="left")
    
    return join_left_df

def point_out_of_bounds(point_df, shape_df,file_dict):
    '''
    Drops points that aren't in the bounded area or are Null
    '''
    minx, miny, maxx, maxy = shape_df.total_bounds
    return point_df[(point_df[file_dict['lat']] > miny) & (point_df[file_dict['lat']] < maxy) & (point_df[file_dict['long']] > minx) & (point_df[file_dict['long']] < maxx)] #Drop Null & out of range points
   
def build_new_points(file_dict):
    '''
    Creates dataframe from file to join to shapefile
    '''
    point_df = pd.read_csv(file_dict['location'])
    point_df = gpd.GeoDataFrame(point_df)
    #Add Points to df
    point_list = []
    for index, row in point_df.iterrows():
        point_list.append(geometry.Point(row[file_dict['long']],row[file_dict['lat']]))
    point_df[GEOMETRY] = point_list
    return point_df

In [24]:
init_calls[GEO_ID]

KeyError: 'TRACTCE10'